#02. Neural Network classification with PyTorch

Classification is a problem of predicting whether something is one thing or another (there can be multiple things as the options). Spam or not spam, steak, sushi, or email.

### 1. Make classification data and get it ready

In [4]:
import sklearn

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
from sklearn.data